In [4]:
import os
import rasterio
import numpy as np
import folium
from pyproj import Transformer
import imageio
import geopandas as gpd

# Paths to the files
dem_path = '/Users/jh/Downloads/Washington_Ground_20ft.tif'
boundary_path = '/Users/jh/Downloads/NCFlood_Effective_Washington_SHP/V_E_FLD_HAZ_AR.shp'
flood_hazard_path = '/Users/jh/Downloads/NCFlood_Prelim_Washington_SHP/V_P_FLD_HAZ_LN.shp'
stream_centerline_path = '/Users/jh/Downloads/NCFlood_Effective_Washington_SHP/V_E_STREAMCNTRLINE.shp'
water_feature_path = '/Users/jh/Downloads/Juliasdata/Re_ shp_files/WaterFeauture.shp'
Stormwater_path = '/Users/jh/Downloads/Juliasdata/Re_ shp_files/Stormwater.shp'

# Open the DEM file using rasterio
with rasterio.open(dem_path) as src:
    dem_data = src.read(1)
    no_data_value = src.nodata
    dem_transform = src.transform
    dem_crs = src.crs
    dem_extent = src.bounds

# Mask the NoData values
if no_data_value is not None:
    dem_data = np.ma.masked_equal(dem_data, no_data_value)

# Clip data to a reasonable range
dem_data_clipped = np.clip(dem_data, 0, 70)

# Normalize DEM data for image representation
dem_data_normalized = (dem_data_clipped - dem_data_clipped.min()) / (dem_data_clipped.max() - dem_data_clipped.min())

# Apply colormap
cmap = cm.get_cmap('terrain')
dem_colored = cmap(dem_data_normalized)

# Remove alpha channel and convert to uint8
dem_colored_uint8 = (dem_colored[:, :, :3] * 255).astype(np.uint8)

# Handle transparency
dem_colored_uint8 = np.dstack([dem_colored_uint8, (~dem_data.mask * 255).astype(np.uint8)])

# Save the colorized DEM as an image
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)
dem_image_path = os.path.join(output_dir, 'dem_colored.png')
imageio.imwrite(dem_image_path, dem_colored_uint8)

# Convert extent to lat/lon using pyproj
transformer = Transformer.from_crs(dem_crs, 'epsg:4326', always_xy=True)
min_lon, min_lat = transformer.transform(dem_extent.left, dem_extent.bottom)
max_lon, max_lat = transformer.transform(dem_extent.right, dem_extent.top)

# Read vector data
boundary_data = gpd.read_file(boundary_path)
flood_hazard_data = gpd.read_file(flood_hazard_path)
stream_centerline_data = gpd.read_file(stream_centerline_path)
water_feature_data = gpd.read_file(water_feature_path)
Stormwater_data = gpd.read_file(Stormwater_path)

# Ensure CRS matches
boundary_data = boundary_data.to_crs(dem_crs)
flood_hazard_data = flood_hazard_data.to_crs(dem_crs)
stream_centerline_data = stream_centerline_data.to_crs(dem_crs)
water_feature_data = water_feature_data.to_crs(dem_crs)
Stormwater_data = Stormwater_data.to_crs(dem_crs)

# Create a folium map centered around Washington County
m = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom_start=12)

# Add OpenStreetMap tiles
folium.TileLayer(
    tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
    attr='&copy; <a href="http://osm.org/copyright">OpenStreetMap</a> contributors',
    name='OpenStreetMap',
    overlay=False,
).add_to(m)

# Add the colorized DEM image as an overlay
img_overlay = folium.raster_layers.ImageOverlay(
    name='DEM',
    image=dem_image_path,
    bounds=[[min_lat, min_lon], [max_lat, max_lon]],
    opacity=0.6,
)
img_overlay.add_to(m)

# Add vector data as GeoJSON overlays with no fill
folium.GeoJson(boundary_data, name='Boundary', style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0}).add_to(m)

# Add flood hazard boundary data
flood_hazard_boundary = flood_hazard_data.boundary
flood_hazard_boundary_json = flood_hazard_boundary.to_json()
folium.GeoJson(flood_hazard_boundary_json, name='Flood Hazard Area', style_function=lambda x: {'color': 'red', 'weight': 1, 'dashArray': '5, 5'}).add_to(m)
folium.GeoJson(stream_centerline_data, name='Stream Centerline', style_function=lambda x: {'color': 'cyan', 'weight': 2}).add_to(m)
folium.GeoJson(water_feature_data, name='Water Feature', style_function=lambda x: {'color': 'green', 'weight': 2}).add_to(m)
folium.GeoJson(Stormwater_data, name='Stormwater', style_function=lambda x: {'color': 'black', 'weight': 2}).add_to(m)

# Add zoomed-in PNG as overlay
zoomed_image_path = os.path.join(output_dir, 'flood_hazard_plot_creswell_zoomed.png')
xmin, ymin, xmax, ymax = 2769000, 778000, 2776000, 784000

# Convert extent to lat/lon using pyproj for the Creswell area
min_lon_creswell, min_lat_creswell = transformer.transform(xmin, ymin)
max_lon_creswell, max_lat_creswell = transformer.transform(xmax, ymax)

zoomed_img_overlay = folium.raster_layers.ImageOverlay(
    name='Zoomed Flood Hazard Area',
    image=zoomed_image_path,
    bounds=[[min_lat_creswell, min_lon_creswell], [max_lat_creswell, max_lon_creswell]],
    opacity=0.6,
)
zoomed_img_overlay.add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
map_output_path = os.path.join(output_dir, 'map_with_dem_vectors_Juliasdata1.html')
m.save(map_output_path)

print(f"Map saved to {map_output_path}")


NameError: name 'cm' is not defined